# 1. let's calculate the density of dry air

# 2. let's get some real data

# 3. let's learn a bit more about water at saturation

In [10]:
# coefficients from Flatau et al. 1992
# doi:10.1175/1520-0450(1992)031<1507%3APFTSVP>2.0.CO%3B2

c_w = (
    6.115836990e000,
    0.444606896e000,
    0.143177157e-01,
    0.264224321e-03,
    0.299291081e-05,
    0.203154182e-07,
    0.702620698e-10,
    0.379534310e-13,
    -.321582393e-15
)

c_i = (
    6.098689930e000,
    0.499320233e000,
    0.184672631e-01,
    0.402737184e-03,
    0.565392987e-05,
    0.521693933e-07,
    0.307839583e-09,
    0.105785160e-11,
    0.161444444e-14
)

# 4. let's then plot the sounding data

# 5. bonus: boiling point